In [1]:
import io
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
path = 'emails/'
for root, folder, filename in os.walk(path):
    print(root)
#     print(filename)

emails/
emails/ham
emails/spam


In [9]:
file_1 = 'emails/ham/00002.9c4069e25e1ef370c078db7ee85ff9ac'
inBody = False
lines = []
f = io.open(file_1,'r')
for line in f:
#     print(line)
    if inBody == True:
        lines.append(line)
#         print(lines)
    elif line == "\n":
        inBody = True
f.close()

In [7]:
lines

['Martin A posted:\n',
 'Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the\n',
 ' limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the\n',
 ' Mount Athos monastic community, was ideal for the patriotic sculpture. \n',
 ' \n',
 " As well as Alexander's granite features, 240 ft high and 170 ft wide, a\n",
 ' museum, a restored amphitheatre and car park for admiring crowds are\n',
 'planned\n',
 '---------------------\n',
 'So is this mountain limestone or granite?\n',
 "If it's limestone, it'll weather pretty fast.\n",
 '\n',
 '------------------------ Yahoo! Groups Sponsor ---------------------~-->\n',
 '4 DVDs Free +s&p Join Now\n',
 'http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM\n',
 '---------------------------------------------------------------------~->\n',
 '\n',
 'To unsubscribe from this group, send an email to:\n',
 'forteana-unsubscribe@egroups.com\n',
 '\n',
 ' \n',
 '\n',
 'Your use of Yahoo! Groups is subject to http:

In [10]:
message = '/n'.join(lines)

In [11]:
message

"Martin A posted:\n/nTassos Papadopoulos, the Greek sculptor behind the plan, judged that the\n/n limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the\n/n Mount Athos monastic community, was ideal for the patriotic sculpture. \n/n \n/n As well as Alexander's granite features, 240 ft high and 170 ft wide, a\n/n museum, a restored amphitheatre and car park for admiring crowds are\n/nplanned\n/n---------------------\n/nSo is this mountain limestone or granite?\n/nIf it's limestone, it'll weather pretty fast.\n/n\n/n------------------------ Yahoo! Groups Sponsor ---------------------~-->\n/n4 DVDs Free +s&p Join Now\n/nhttp://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM\n/n---------------------------------------------------------------------~->\n/n\n/nTo unsubscribe from this group, send an email to:\n/nforteana-unsubscribe@egroups.com\n/n\n/n \n/n\n/nYour use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/ \n/n\n/n\n/n\n"

In [14]:
def read_data(path):
    for root, folder, filename in os.walk(path):
        for name in filename:
            file = os.path.join(root,name)
    #         print(file)
            inBody = False
            lines = []
            f = io.open(file,'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = "\n".join(lines)
            yield message, path        

In [15]:
def df(path, classification):
    rows = []
    index = []
    for message, filename in read_data(path):
        rows.append({'message':message,'class':classification})
        index.append(filename)
        
    return pd.DataFrame(data=rows, index=index)

dataset = pd.DataFrame({'message':[], 'class':[]})
dataset = dataset.append(df('emails/ham', 'ham'))
dataset = dataset.append(df('emails/spam', 'spam'))

In [16]:
dataset.head()

,class,message
emails/ham,ham,"Date: Wed, 21 Aug 2002 10:54:46 -05..."
emails/ham,ham,"Martin A posted:\n\nTassos Papadopoulos, the G..."
emails/ham,ham,Man Threatens Explosion In Moscow \n\n\n\nThur...
emails/ham,ham,Klez: The Virus That Won't Die\n\n \n\nAlready...
emails/ham,ham,"> in adding cream to spaghetti carbonara, whi..."


In [17]:
dataset.tail()

,class,message
emails/spam,spam,<html>\n\n<head>\n\n<title>Toy</title>\n\n</he...
emails/spam,spam,<html>\n\n<head>\n\n<title>Untitled Document</...
emails/spam,spam,This is an HTML email message. If you see thi...
emails/spam,spam,<html>\n\n<head>\n\n</head>\n\n <body backgro...
emails/spam,spam,"<STYLE type=""text/css"">\n\n<!--\n\nP{\n\n fon..."


In [18]:
dataset.shape

(3000, 2)

In [19]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(dataset['message'].values)

classifier = MultinomialNB()
targets = dataset['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
examples = ['hello there...Hope you doing well',
            'Win free movie tickets and offers']
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['ham', 'spam'],
      dtype='<U4')